In [14]:
import os
import pdfplumber
import pandas as pd
import promptlib

In [15]:
#Procura o arquivo na pasta específica
prompter = promptlib.Files()
file = prompter.file()


In [16]:
#Abre o arquivo pdf com pdfplumber
pdf = pdfplumber.open(file)
#Lê a primeira página
page = pdf.pages[0]
#extrai o texto da página
text=page.extract_text()
#Converte o texto em linhas
lines = text.splitlines()
#lines = text.split('\n')
#Cria listas para armazenar cada dado
_nota,_data,_cnpj,_cod,_quant,_unit,_total,_icms,_aliq = [],[],[],[],[],[],[],[],[]

In [17]:
#Faz um loop em lines para capturar os dados
for (i,item) in enumerate(lines):
    #Captura cabeçário da Nota Fiscal
    if item=='DESTINATÁRIO: NF-e Associacao NF-e':
        nota = lines[i+1].split(' ')[1]
    if item == 'NOME / RAZÃO SOCIAL CNPJ / CPF DATA DA EMISSÃO':
        empresa = lines[i+1].split(' ')[:-2]
        empresa = ' '.join(empresa)
        cnpj = lines[i+1].split(' ')[-2]
        data = lines[i+1].split(' ')[-1]
    if item =='BASE DE CÁLCULO DO ICMS VALOR DO ICMS BASE DE CÁLCULO DO ICMS S.T. VALOR DO ICMS SUBSTITUIÇÃO VALOR DO PIS VALOR TOTAL DOS PRODUTOS':
        base = lines[i+1].split(' ')[0]
    if item =='CÓDIGO PRODUTO DESCRIÇÃO DO PRODUTO / SERVIÇO NCM/SH O/CST CFOP UN QUANT VUANLIOTR VTOATLAOLR BI.CCMÁLSC VIACLMOSR VAIPLIOR AICLMÍQS. AILPÍIQ.':
        n=1
        l= lines[i+n].split(' ')
        cod= l[0]
        #Captura itens da Nota Fiscal
        while cod.isnumeric():
            quant=l[-5]
            unit=l[-4]
            total = l[-3]
            icms = l[-2]
            aliq=l[-1]
            #print(nota,data,cnpj,cod,quant,unit,total,icms,aliq)
            _quant.append(quant)
            _unit.append(unit)
            _total.append(total)
            _icms.append(icms)
            _aliq.append(aliq)
            _cnpj.append(cnpj)
            _nota.append(nota)
            _data.append(data)
            _cod.append(cod)
            n+=1
            l= lines[i+n].split(' ')
            cod= l[0]

In [18]:
#Salva os dados em um dicionário e depois converte para um DataFrame
dict={"Nota":_nota,"Data":_data,"CNPJ":_cnpj,"Cod_Prod":_cod,"Quant.":_quant,"Vlr_Unit":_unit,"Vlr_Total":_total,"ICMS":_icms,"Aliq":_aliq}
df=pd.DataFrame.from_dict(dict,orient='index').transpose()
df

,Nota,Data,CNPJ,Cod_Prod,Quant.,Vlr_Unit,Vlr_Total,ICMS,Aliq
0,000.000.001,06/05/2008,00.000.000/0001-91,00001,"1.000.000,00","1,0000","10.000.000,00","10.000.000,001.800.000,00",18
1,000.000.001,06/05/2008,00.000.000/0001-91,00002,"5.000.000,00","2,0000","10.000.000,00","10.000.000,001.800.000,00",18
